# Diffusion coefficients from MSD

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mdtraj

ATTENTION: Make sure the trajectory was obtained with `enforcePeriodicBox=False` in the `DCDReporter`, otherwise coordinates will be wrapped

In [ ]:
trj = mdtraj.load('traj.dcd', top='config.pdb')
trj

In [ ]:
help(trj.topology.to_dataframe)

In [ ]:
top = trj.topology
table, bonds = top.to_dataframe()

In [ ]:
print(table)

In [ ]:
print(bonds)

In [ ]:
print((table['resName'] == 'SPC') & (table['element'] == 'O'))

In [ ]:
atoms = table[(table['resName'] == 'SPC') & (table['element'] == 'O')].index
atoms

Our own function to calculate MSD

In [ ]:
def calcmsd(trj, idx):
    """ Compute MSD(t)
    trj - mdtraj trajectory
    idx - 1D array with atom indices to consider 
    """
    msd = np.zeros(trj.n_frames)
    for i in range(trj.n_frames):
        disp = trj.xyz[i, idx] - trj.xyz[0, idx]
        msd[i] = np.sum(disp**2)/len(idx)
    return msd

In [ ]:
mymsd = calcmsd(trj, atoms)

Atom selection using mdtraj

In [ ]:
atoms2 = trj.topology.select("resname SPC and element O")
atoms2, len(atoms2)

MSD from MDTraj

In [ ]:
rmsd = mdtraj.rmsd(trj, trj, frame=0, atom_indices=None)
msd = rmsd**2

In [ ]:
t = np.linspace(0, 500, 1000)

In [ ]:
plt.plot(t, msd, label='H2O')
plt.plot(t, mymsd, label='H2O my')
plt.xlabel('t / ps')
plt.ylabel('MSD / nm$^2$')
plt.legend()
plt.show()

In [ ]:
np.savetxt('msd.out', np.column_stack((t, msd)))

In [ ]:
(D, b) = np.polyfit(t[50:], msd[50:], deg=1)

In [ ]:
D/6/1e6

In [ ]:
plt.plot(t, msd, label='H2O')
plt.plot(t, D*t + b, 'b-')
plt.xlabel('t / ps')
plt.ylabel('MSD / nm$^2$')
plt.legend()
plt.show()